In [1]:
import os
import pickle
import argparse

import gensim
from gensim.models import word2vec

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import nltk
from nltk.tokenize import word_tokenize


from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.models import Sequential
from keras.optimizers import Adam
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.utils import to_categorical
#x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
from keras import backend as K
from keras.callbacks import EarlyStopping


Using TensorFlow backend.


In [2]:
model = Sequential()
model.add(LSTM(128,input_shape=(398,40),activation='sigmoid', inner_activation='hard_sigmoid', return_sequences=True))
model.add(Activation('sigmoid'))
model.add(Dropout(0.2))
#model.add(TimeDistributedDense(11))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

/home/mygpu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, return_sequences=True, input_shape=(398, 40), activation="sigmoid", recurrent_activation="hard_sigmoid")`
  from ipykernel import kernelapp as app


In [3]:
def pad_sequences(sequences, maxlen=None, dim=1, dtype='int32',
    padding='pre', truncating='pre', value=0.):
    '''
        Override keras method to allow multiple feature dimensions.

        @dim: input feature dimension (number of features per timestep)
    '''
    lengths = [len(s) for s in sequences]

    nb_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    x = (np.ones((nb_samples, maxlen, dim)) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError("Truncating type '%s' not understood" % padding)

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError("Padding type '%s' not understood" % padding)
    return (x)

In [4]:
from keras.preprocessing.text import Tokenizer#,base_filter
from keras.preprocessing.sequence import pad_sequences

def shift(seq, n):
    n = n % len(seq)
    return (seq[n:] + seq[:n])

txt="abcdefghijklmn"*100

tk = Tokenizer(nb_words=2000, lower=True, split=" ")
tk.fit_on_texts(txt)
x = tk.texts_to_sequences(txt)
#shifing to left
y = shift(x,1)

#padding sequence
max_len = 100
max_features=len(tk.word_counts)
X = pad_sequences(x, maxlen=max_len)
Y = pad_sequences(y, maxlen=max_len)

/home/mygpu/anaconda3/lib/python3.5/site-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [5]:
X

array([[ 0,  0,  0, ...,  0,  0,  1],
       [ 0,  0,  0, ...,  0,  0,  2],
       [ 0,  0,  0, ...,  0,  0,  3],
       ..., 
       [ 0,  0,  0, ...,  0,  0, 12],
       [ 0,  0,  0, ...,  0,  0, 13],
       [ 0,  0,  0, ...,  0,  0, 14]], dtype=int32)

In [6]:
'''Trains an LSTM model on the IMDB sentiment classification task.
The dataset is actually too small for LSTM to be of any advantage
compared to simpler, much faster methods such as TF-IDF + LogReg.
Notes:
- RNNs are tricky. Choice of batch size is important,
choice of loss and optimizer is critical, etc.
Some configurations won't converge.
- LSTM loss decrease patterns during training can be quite different
from what you see with CNNs/MLPs/etc.
'''
from __future__ import print_function

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb

max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

print('Loading data...')
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')

print('Pad sequences (samples x time)')
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=3,
          validation_data=(x_test, y_test))
score, acc = model.evaluate(x_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Loading data...
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
x_train shape: (25000, 80)
x_test shape: (25000, 80)
Build model...
Train...
Train on 25000 samples, validate on 25000 samples
Epoch 1/3
25000/25000 [==============================] - 113s - loss: 0.4621 - acc: 0.7769 - val_loss: 0.3976 - val_acc: 0.8348
Epoch 2/3
25000/25000 [==============================] - 106s - loss: 0.2933 - acc: 0.8797 - val_loss: 0.3862 - val_acc: 0.8398
Epoch 3/3
25000/25000 [==============================] - 19s    
Test score: 0.418655139561
Test accuracy: 0.82752


In [7]:
X_data = [[1,2,3,4,5,6,7,8,9,10],
         [343,43,3,4,5,6,7,8,9,10]]

In [8]:
X_data

[[1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [343, 43, 3, 4, 5, 6, 7, 8, 9, 10]]

In [9]:
x_train = sequence.pad_sequences(X_data, maxlen=10)
x_test = sequence.pad_sequences([[2,4,6,8,9],
         [343,43,8,9,10]],maxlen=10)

In [10]:
x_test

array([[  0,   0,   0,   0,   0,   2,   4,   6,   8,   9],
       [  0,   0,   0,   0,   0, 343,  43,   8,   9,  10]], dtype=int32)

In [11]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import

import numpy as np
import random
from six.moves import range


def pad_sequences(sequences, maxlen=None, dtype='int32',
                  padding='pre', truncating='pre', value=0.):
    """Pads each sequence to the same length (length of the longest sequence).
    If maxlen is provided, any sequence longer
    than maxlen is truncated to maxlen.
    Truncation happens off either the beginning (default) or
    the end of the sequence.
    Supports post-padding and pre-padding (default).
    # Arguments
        sequences: list of lists where each element is a sequence
        maxlen: int, maximum length
        dtype: type to cast the resulting sequence.
        padding: 'pre' or 'post', pad either before or after each sequence.
        truncating: 'pre' or 'post', remove values from sequences larger than
            maxlen either in the beginning or in the end of the sequence
        value: float, value to pad the sequences to the desired value.
    # Returns
        x: numpy array with dimensions (number_of_sequences, maxlen)
    # Raises
        ValueError: in case of invalid values for `truncating` or `padding`,
            or in case of invalid shape for a `sequences` entry.
    """
    if not hasattr(sequences, '__len__'):
        raise ValueError('`sequences` must be iterable.')
    lengths = []
    for x in sequences:
        if not hasattr(x, '__len__'):
            raise ValueError('`sequences` must be a list of iterables. '
                             'Found non-iterable: ' + str(x))
        lengths.append(len(x))

    num_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((num_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if not len(s):
            continue  # empty list/array was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x


def make_sampling_table(size, sampling_factor=1e-5):
    """Generates a word rank-based probabilistic sampling table.
    This generates an array where the ith element
    is the probability that a word of rank i would be sampled,
    according to the sampling distribution used in word2vec.
    The word2vec formula is:
        p(word) = min(1, sqrt(word.frequency/sampling_factor) / (word.frequency/sampling_factor))
    We assume that the word frequencies follow Zipf's law (s=1) to derive
    a numerical approximation of frequency(rank):
       frequency(rank) ~ 1/(rank * (log(rank) + gamma) + 1/2 - 1/(12*rank))
        where gamma is the Euler-Mascheroni constant.
    # Arguments
        size: int, number of possible words to sample.
        sampling_factor: the sampling factor in the word2vec formula.
    # Returns
        A 1D Numpy array of length `size` where the ith entry
        is the probability that a word of rank i should be sampled.
    """
    gamma = 0.577
    rank = np.arange(size)
    rank[0] = 1
    inv_fq = rank * (np.log(rank) + gamma) + 0.5 - 1. / (12. * rank)
    f = sampling_factor * inv_fq

    return np.minimum(1., f / np.sqrt(f))


def skipgrams(sequence, vocabulary_size,
              window_size=4, negative_samples=1., shuffle=True,
              categorical=False, sampling_table=None, seed=None):
    """Generates skipgram word pairs.
    Takes a sequence (list of indexes of words),
    returns couples of [word_index, other_word index] and labels (1s or 0s),
    where label = 1 if 'other_word' belongs to the context of 'word',
    and label=0 if 'other_word' is randomly sampled
    # Arguments
        sequence: a word sequence (sentence), encoded as a list
            of word indices (integers). If using a `sampling_table`,
            word indices are expected to match the rank
            of the words in a reference dataset (e.g. 10 would encode
            the 10-th most frequently occurring token).
            Note that index 0 is expected to be a non-word and will be skipped.
        vocabulary_size: int. maximum possible word index + 1
        window_size: int. actually half-window.
            The window of a word wi will be [i-window_size, i+window_size+1]
        negative_samples: float >= 0. 0 for no negative (=random) samples.
            1 for same number as positive samples. etc.
        shuffle: whether to shuffle the word couples before returning them.
        categorical: bool. if False, labels will be
            integers (eg. [0, 1, 1 .. ]),
            if True labels will be categorical eg. [[1,0],[0,1],[0,1] .. ]
        sampling_table: 1D array of size `vocabulary_size` where the entry i
            encodes the probability to sample a word of rank i.
        seed: random seed.
    # Returns
        couples, labels: where `couples` are int pairs and
            `labels` are either 0 or 1.
    # Note
        By convention, index 0 in the vocabulary is
        a non-word and will be skipped.
    """
    couples = []
    labels = []
    for i, wi in enumerate(sequence):
        if not wi:
            continue
        if sampling_table is not None:
            if sampling_table[wi] < random.random():
                continue

        window_start = max(0, i - window_size)
        window_end = min(len(sequence), i + window_size + 1)
        for j in range(window_start, window_end):
            if j != i:
                wj = sequence[j]
                if not wj:
                    continue
                couples.append([wi, wj])
                if categorical:
                    labels.append([0, 1])
                else:
                    labels.append(1)

    if negative_samples > 0:
        num_negative_samples = int(len(labels) * negative_samples)
        words = [c[0] for c in couples]
        random.shuffle(words)

        couples += [[words[i % len(words)],
                    random.randint(1, vocabulary_size - 1)] for i in range(num_negative_samples)]
        if categorical:
            labels += [[1, 0]] * num_negative_samples
        else:
            labels += [0] * num_negative_samples

    if shuffle:
        if seed is None:
            seed = random.randint(0, 10e6)
        random.seed(seed)
        random.shuffle(couples)
        random.seed(seed)
        random.shuffle(labels)

    return couples, labels


def _remove_long_seq(maxlen, seq, label):
    """Removes sequences that exceed the maximum length.
    # Arguments
        maxlen: int, maximum length
        seq: list of lists where each sublist is a sequence
        label: list where each element is an integer
    # Returns
        new_seq, new_label: shortened lists for `seq` and `label`.
    """
    new_seq, new_label = [], []
    for x, y in zip(seq, label):
        if len(x) < maxlen:
            new_seq.append(x)
            new_label.append(y)
    return new_seq, new_label

In [42]:
def sliding_window(df,window_size,window_stride,input_features=None, output_features=None):
    allData = df[input_features].values.tolist()
    if output_features:
        outputData = df[output_features].values.tolist()
    print(outputData)
    myArray = [[]]
    outputDataArray = [] 
    start=0
    for i in range(0,len(allData),window_stride):
        #print(allData[i:window_size+i])
        print(i)
        if i == 0:
            myArray = [allData[i:window_size+i]]
            if output_features != None:
                outputDataArray.append(outputData[window_size+i][0])
        else:
            myArray.append(allData[i:window_size+i])
            
            if window_size+i >= len(allData):
                if output_features != None:
                    outputDataArray.append(outputData[len(allData)-1][0])
                break
            if output_features != None:
                    outputDataArray.append(outputData[window_size+i][0])

    if output_features == None:
            return (np.array(myArray))
    print(outputDataArray)   
    return (np.array(myArray), outputDataArray)

In [43]:
import pandas as pd
import numpy as np
# Get some time series data
df = pd.read_csv("https://raw.githubusercontent.com/plotly/datasets/master/timeseries.csv")
df['output']=[1,0,1,0,1,0,1,0,1,0,1]

In [44]:
df['output']

0     1
1     0
2     1
3     0
4     1
5     0
6     1
7     0
8     1
9     0
10    1
Name: output, dtype: int64

In [45]:
input_cols = ['A','B','C']
x_train,y_train=sliding_window(df,5,2,input_cols,['output'])

[[1], [0], [1], [0], [1], [0], [1], [0], [1], [0], [1]]
0
2
4
6
[0, 0, 0, 1]


In [46]:
x_train

array([[[  24.68,  164.93,  114.73],
        [  24.18,  164.89,  114.75],
        [  23.99,  164.63,  115.04],
        [  24.14,  163.92,  114.85],
        [  24.44,  163.45,  114.84]],

       [[  23.99,  164.63,  115.04],
        [  24.14,  163.92,  114.85],
        [  24.44,  163.45,  114.84],
        [  24.38,  163.46,  115.4 ],
        [  24.32,  163.22,  115.56]],

       [[  24.44,  163.45,  114.84],
        [  24.38,  163.46,  115.4 ],
        [  24.32,  163.22,  115.56],
        [  24.19,  164.02,  115.54],
        [  23.81,  163.59,  115.72]],

       [[  24.32,  163.22,  115.56],
        [  24.19,  164.02,  115.54],
        [  23.81,  163.59,  115.72],
        [  24.03,  163.32,  115.11],
        [  24.34,  163.34,  115.17]]])

In [72]:
#y_train = np.array(y_train)
y_train = np.array(y_train).reshape((-1, 1))
y_train = to_categorical(y_train)

In [73]:
y_train

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.]])

In [77]:
model = Sequential()
# input_shape = number of time-steps, number-of-features
model.add(LSTM(128,input_shape=(5,3),
               activation='sigmoid', 
               inner_activation='hard_sigmoid', 
               return_sequences=True))
model.add(LSTM(128, activation='tanh', recurrent_activation='hard_sigmoid'))
#model.add(Activation('sigmoid'))
model.add(Dropout(0.2))
#model.add(TimeDistributedDense(11))
#model.add(Dense(128))
model.add(Dense(64, kernel_initializer='uniform', activation='relu'))
model.add(Dense(output_dim=2, kernel_initializer='uniform', activation='sigmoid'))
#model.add(Activation('sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

/home/mygpu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(128, return_sequences=True, input_shape=(5, 3), activation="sigmoid", recurrent_activation="hard_sigmoid")`
/home/mygpu/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=2, activation="sigmoid", kernel_initializer="uniform")`


In [78]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 5, 128)            67584     
_________________________________________________________________
lstm_16 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_12 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 130       
Total params: 207,554
Trainable params: 207,554
Non-trainable params: 0
_________________________________________________________________


In [85]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=1,
          epochs=5,
          validation_data=(x_train, y_train))
#score, acc = 


Train...
Train on 4 samples, validate on 4 samples
Epoch 1/5
4/4 [==============================] - 0s - loss: 0.5973 - val_loss: 0.5623
Epoch 2/5
4/4 [==============================] - 0s - loss: 0.5749 - val_loss: 0.5623
Epoch 3/5
4/4 [==============================] - 0s - loss: 0.5987 - val_loss: 0.5624
Epoch 4/5
4/4 [==============================] - 0s - loss: 0.5800 - val_loss: 0.5624
Epoch 5/5
4/4 [==============================] - 0s - loss: 0.5977 - val_loss: 0.5624


In [87]:
score = model.evaluate(x_train, y_train,batch_size=1)
print()
print('Test score:', score)
print('Test accuracy:', acc)

1/4 [======>.......................] - ETA: 0s
Test score: 0.562384031713
Test accuracy: 0.82752


In [88]:
model.predict(x_train)

array([[ 0.74791425,  0.24362557],
       [ 0.74791294,  0.2436263 ],
       [ 0.74791336,  0.24362645],
       [ 0.74791366,  0.24362709]], dtype=float32)

1.0

array([[  24.68,  164.93,  114.73],
       [  24.18,  164.89,  114.75],
       [  23.99,  164.63,  115.04],
       [  24.14,  163.92,  114.85],
       [  24.44,  163.45,  114.84]])

array([[  24.18,  164.89,  114.75],
       [  23.99,  164.63,  115.04],
       [  24.14,  163.92,  114.85],
       [  24.44,  163.45,  114.84],
       [  24.38,  163.46,  115.4 ]])

array([[[  24.18,  164.89,  114.75],
        [  23.99,  164.63,  115.04],
        [  24.14,  163.92,  114.85],
        [  24.44,  163.45,  114.84],
        [  24.38,  163.46,  115.4 ]]])

[array([[  24.18,  164.89,  114.75],
        [  23.99,  164.63,  115.04],
        [  24.14,  163.92,  114.85],
        [  24.44,  163.45,  114.84],
        [  24.38,  163.46,  115.4 ]])]

[[[]], array([[  24.18,  164.89,  114.75],
        [  23.99,  164.63,  115.04],
        [  24.14,  163.92,  114.85],
        [  24.44,  163.45,  114.84],
        [  24.38,  163.46,  115.4 ]]), [[24.68, 164.93, 114.73],
  [24.18, 164.89, 114.75],
  [23.99, 164.63, 115.04],
  [24.14, 163.92, 114.85],
  [24.44, 163.45, 114.84],
  [24.38, 163.46, 115.4],
  [24.32, 163.22, 115.56],
  [24.19, 164.02, 115.54],
  [23.81, 163.59, 115.72],
  [24.03, 163.32, 115.11],
  [24.34, 163.34, 115.17]]]

In [87]:
ALL

[[24.68, 164.93, 114.73],
 [24.18, 164.89, 114.75],
 [23.99, 164.63, 115.04],
 [24.14, 163.92, 114.85],
 [24.44, 163.45, 114.84],
 [24.38, 163.46, 115.4],
 [24.32, 163.22, 115.56],
 [24.19, 164.02, 115.54],
 [23.81, 163.59, 115.72],
 [24.03, 163.32, 115.11],
 [24.34, 163.34, 115.17]]

In [88]:
a=[[[]]]

In [89]:
a[0] = ALL[0:5]

In [90]:
a.append(ALL[1:6])

In [91]:
a

[[[24.68, 164.93, 114.73],
  [24.18, 164.89, 114.75],
  [23.99, 164.63, 115.04],
  [24.14, 163.92, 114.85],
  [24.44, 163.45, 114.84]],
 [[24.18, 164.89, 114.75],
  [23.99, 164.63, 115.04],
  [24.14, 163.92, 114.85],
  [24.44, 163.45, 114.84],
  [24.38, 163.46, 115.4]]]

In [92]:
np.array(a)

array([[[  24.68,  164.93,  114.73],
        [  24.18,  164.89,  114.75],
        [  23.99,  164.63,  115.04],
        [  24.14,  163.92,  114.85],
        [  24.44,  163.45,  114.84]],

       [[  24.18,  164.89,  114.75],
        [  23.99,  164.63,  115.04],
        [  24.14,  163.92,  114.85],
        [  24.44,  163.45,  114.84],
        [  24.38,  163.46,  115.4 ]]])

In [97]:
len(ALL)

11